In [1]:
import re
import os, json
import warnings
from datetime import datetime
from pprint import pprint
from textwrap import dedent
from dotenv import load_dotenv
from langchain_core.messages import ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig

warnings.filterwarnings("ignore")
load_dotenv()

True

In [2]:
from langchain_community.tools import TavilySearchResults

query = "최근 한 달 동안 나온 테슬라와 구글 CEO 관련 영문 기사들을 각각 자연스럽게 한국어로 요약해서 Bullet Point로 정리해줘"

tavily_search = TavilySearchResults(max_results=2)
search_results = tavily_search.invoke(query)
search_results

[{'title': '구글 CEO의 고백, 테슬라가 최고다! 이유는? 인터뷰 번역, 댄 아이브스 ...',
  'url': 'https://www.youtube.com/watch?v=nu_9tQXlFBQ',
  'content': '구글 CEO의 고백, 테슬라가 최고다! 이유는? 인터뷰 번역, 댄 아이브스 예상주가 $515 간다! 테슬라 음성 서비스 시작하나  ? \n 오늘의 테슬라 뉴스 \n 3370 likes \n 57618 views \n 16 Dec 2024 \n ➤ 테슬라 구매하시나요? 추천링크 이동해서 33만원 할인을 받으세요.  ( 미국 $2천 달러 할인 )\n      추천링크 https://ts.la/sung85917\n\n➤ 오늘의 테슬라 뉴스 트위터 https://twitter.com/TodayTeslanews\n\n➤ 오늘의 테슬라 뉴스 회원가입https://www.youtube.com/c/%EC%98%A4%EB%8A%98%EC%9D%98%ED%85%8C%EC%8A%AC%EB%9D%BC%EB%89%B4%EC%8A%A4/join\n\n➤ Surfshark + 오테뉴 이벤트 4개월 추가 제공과 최저가 해택 놓치지 마세요! \n     https://surfshark.com/teslanews [...] ➤ 오늘의 테슬라 뉴스 광고 문의  todayteslanews@gmail.com\n\n➤욕설과 악플, 도배댓글, 타채널 홍보링크, 매너없는 글은 삭제/차단 합니다.\n➤주가 예측 질문 (살까요, 팔까요) 댓글에는 답변하지 않습니다.\n\n#테슬라 #테슬라주식 #테슬라뉴스 #ARK #일론머스크 #모델3  #모델y #investing #투자 #elonmusk #TSLA #Tesla #오테뉴 #트위터 #스페이스X #Twitter #오늘의테슬라뉴스 #주식투자 #해외주식 \n 235 comments',
  'score': 0.5852203},
 {'title': '[시사영어 1일1문] 머스크의 테슬라 사면초가에 (NYT ... - YouTube',
  '

In [3]:
TavilySearchResults # class 살펴보기
type(tavily_search)

langchain_community.tools.tavily_search.tool.TavilySearchResults

In [4]:
tavily_search.name

'tavily_search_results_json'

In [5]:
tavily_search.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [6]:
tavily_search.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [7]:
tavily_search.args_schema

langchain_community.tools.tavily_search.tool.TavilyInput

In [8]:
tavily_search.args_schema.model_json_schema()

{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[tavily_search])

query

'최근 한 달 동안 나온 테슬라와 구글 CEO 관련 영문 기사들을 각각 자연스럽게 한국어로 요약해서 Bullet Point로 정리해줘'

In [10]:
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3UzLuq6NwfyaTWXZ5WtAEuKl', 'function': {'arguments': '{"query": "Tesla CEO news articles last month"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_mCuBtgA9omnkWKnoE7aI3etb', 'function': {'arguments': '{"query": "Google CEO news articles last month"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 113, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BQYJa3UmF8Wbvl3gW2l7k2DMu6RiS', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-522347aa-20e2-4975-b3bb-98c768f368fe-0', tool_calls=[{'name': 'tavily_search

In [11]:
ai_msg.content

''

In [12]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'Tesla CEO news articles last month'},
  'id': 'call_3UzLuq6NwfyaTWXZ5WtAEuKl',
  'type': 'tool_call'},
 {'name': 'tavily_search_results_json',
  'args': {'query': 'Google CEO news articles last month'},
  'id': 'call_mCuBtgA9omnkWKnoE7aI3etb',
  'type': 'tool_call'}]

In [13]:
# args 스키마 사용
tool_call = ai_msg.tool_calls[0]
tool_output = tavily_search.invoke(tool_call['args'])
pprint(type(tool_output))
pprint(tool_output)

<class 'list'>
[{'content': 'Watch CBS News\n'
             '\n'
             "As Tesla's sales and profit tumble, investors ask Elon Musk to "
             'ditch DOGE and return to automaker\n'
             '\n'
             'Updated on:  April 22, 2025 / 6:35 PM EDT\n'
             '          / CBS News\n'
             '\n'
             'Tesla CEO Elon Musk is facing a corporate reckoning on Tuesday, '
             "with the electric car maker's first-quarter results showing "
             'sales and profit fell short of analyst expectations while the '
             "billionaire was focused on leading the Trump administration's "
             'cost-cutting efforts. [...] Earlier this month, Tesla said its '
             'delivery of vehicles in the first three months of 2025 fell '
             "nearly 13% as demand for the automaker's electric cars continued "
             "to weaken, with deliveries also falling well short of analysts' "
             "expectations. [...] In additi

In [14]:
# tool_call 사용
tool_message = tavily_search.invoke(tool_call)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content='[{"title": "As Tesla\'s sales and profit tumble, investors ask Elon Musk to ditch ...", "url": "https://www.cbsnews.com/news/tesla-earnings-call-today-elon-musk-doge/", "content": "Watch CBS News\\n\\nAs Tesla\'s sales and profit tumble, investors ask Elon Musk to ditch DOGE and return to automaker\\n\\nUpdated on:  April 22, 2025 / 6:35 PM EDT\\n          / CBS News\\n\\nTesla CEO Elon Musk is facing a corporate reckoning on Tuesday, with the electric car maker\'s first-quarter results showing sales and profit fell short of analyst expectations while the billionaire was focused on leading the Trump administration\'s cost-cutting efforts. [...] Earlier this month, Tesla said its delivery of vehicles in the first three months of 2025 fell nearly 13% as demand for the automaker\'s electric cars continued to weaken, with deliveries also falling well short of analysts\' expectations.", "score": 0.7154444}, {"title": "E

In [15]:
# tool_output으로 ToolMessage 만들기
from langchain_core.messages.tool import ToolMessage

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content=[{'title': "As Tesla's sales and profit tumble, investors ask Elon Musk to ditch ...", 'url': 'https://www.cbsnews.com/news/tesla-earnings-call-today-elon-musk-doge/', 'content': "Watch CBS News\n\nAs Tesla's sales and profit tumble, investors ask Elon Musk to ditch DOGE and return to automaker\n\nUpdated on:  April 22, 2025 / 6:35 PM EDT\n          / CBS News\n\nTesla CEO Elon Musk is facing a corporate reckoning on Tuesday, with the electric car maker's first-quarter results showing sales and profit fell short of analyst expectations while the billionaire was focused on leading the Trump administration's cost-cutting efforts. [...] Earlier this month, Tesla said its delivery of vehicles in the first three months of 2025 fell nearly 13% as demand for the automaker's electric cars continued to weaken, with deliveries also falling well short of analysts' expectations. [...] In addition to insights into Musk's role at

In [16]:
tool_message.content

[{'title': "As Tesla's sales and profit tumble, investors ask Elon Musk to ditch ...",
  'url': 'https://www.cbsnews.com/news/tesla-earnings-call-today-elon-musk-doge/',
  'content': "Watch CBS News\n\nAs Tesla's sales and profit tumble, investors ask Elon Musk to ditch DOGE and return to automaker\n\nUpdated on:  April 22, 2025 / 6:35 PM EDT\n          / CBS News\n\nTesla CEO Elon Musk is facing a corporate reckoning on Tuesday, with the electric car maker's first-quarter results showing sales and profit fell short of analyst expectations while the billionaire was focused on leading the Trump administration's cost-cutting efforts. [...] Earlier this month, Tesla said its delivery of vehicles in the first three months of 2025 fell nearly 13% as demand for the automaker's electric cars continued to weaken, with deliveries also falling well short of analysts' expectations. [...] In addition to insights into Musk's role at Tesla, investors will also be listening for updates on several str

In [17]:
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = f"""
    You are a helpful AI assistant.
    Today is {today}
    {query}
    {tool_message.content}
"""
result = llm.invoke(prompt)
pprint(result)

AIMessage(content='### 테슬라 CEO 엘론 머스크 관련 요약\n\n- 테슬라의 1분기 실적이 예상치를 밑돌며 매출과 이익이 감소하자, 투자자들은 머스크가 DOGE(도지코인)에 대한 집중을 줄이고 자동차 제조에 더 집중할 것을 요구하고 있음.\n- 최근 테슬라는 2025년 1분기 차량 납품량이 약 13% 감소했다고 발표, 전반적인 전기차 수요가 줄어드는 상황에서 납품량도 분석가들의 예상에 크게 미치지 못함.\n- 머스크는 올해 후반에 자사의 베스트셀러 차량인 모델 Y SUV의 저렴한 버전을 출시할 계획과 6월에 텍사스 오스틴에서 유료 자율 로봇택시 서비스를 시작할 것이라고 언급함.\n- 머스크는 향후 DOGE 관련 활동에 더 많은 시간을 투자할 계획이었으나, 5월부터는 그 시간이 "상당히" 줄어들 것이라고 밝혔음.\n- 테슬라는 중국의 경쟁과 노후 차량 문제 외에도, 머스크의 트럼프와의 연관으로 인한 브랜드 이미지 손상, 미국 및 유럽에서의 반대 시위 등 여러 도전에 직면해 있음.\n\n### 구글 CEO 관련 최근 기사 없음\n- 최근 한 달간 구글 CEO에 관한 영문 기사는 수집되지 않았습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 314, 'prompt_tokens': 671, 'total_tokens': 985, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352

In [18]:
# LLM 체인 정의

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant"),
    ("system", f"Today is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}")
])

# llm 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search])

# LLM체인 생성
llm_chain = prompt | llm_with_tools


In [19]:
# LLM 체인에 ToolMessage 전달
from langchain_core.runnables import chain

@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input, config=config)
    tool_msgs = tavily_search.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke(
        {**input, "messages": [ai_msg, *tool_msgs]},
        config=config
    )

web_search_chain.invoke(query)

AIMessage(content="### 테슬라 CEO 일론 머스크 관련 최근 기사 요약\n- **DOGE 관련 발언**: 머스크는 2025년 5월부터 자신의 DOGE 관련 시간 투자가 '상당히 줄어들 것'이라고 발표했다. 이는 회사의 실적 발표 전 전화 회의에서 이뤄졌다. ([자세히 보기](https://www.cnbc.com/2025/04/22/tesla-ceo-elon-musk-says-time-he-spends-on-doge-will-drop-significantly-next-month.html))\n- **경쟁과 도전**: 테슬라는 중국의 경쟁과 노후화된 전기차 문제 외에도 미국과 유럽에서의 시위와 머스크의 정치적 연관으로 인한 브랜드 이미지 손상이란 도전에 직면해 있다.\n\n### 구글 CEO 순다르 피차이 관련 최근 기사 요약\n- **2025년 1분기 실적 발표**: 구글과 알파벳의 CEO 피차이는 1분기 실적 발표에서 검색, 클라우드, 유튜브 부문에서의 성장을 강조하며, AI 기술이 새로운 기능을 강화하는 데 중요한 역할을 하고 있다고 설명했다. ([자세히 보기](https://blog.google/inside-google/message-ceo/alphabet-earnings-q1-2025/))\n- **구독 서비스 성장**: 유튜브와 구글 원의 도움이 컸으며, 270백만 개 이상의 유료 구독을 기록했다고 언급했다.\n- **2025년에 대한 경고**: 피차이는 직원들에게 2025년은 경쟁과 규제의 증가로 인해 '위험이 크다'고 경고했다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 369, 'prompt_tokens': 1189, 'total_tokens': 1558, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'r

In [20]:
from typing import Dict, List
import requests
from langchain_core.tools import tool

@tool
def blog_search(query: str) -> List[Dict]:
    """네이버 블로그 API에 검색 요청을 보냅니다."""
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": os.environ["NAVER_CLIENT_ID"],
        "X-Naver-Client-Secret": os.environ["NAVER_CLIENT_SECRET"]
    }
    params = {"query": query, "display": 10, "start": 1}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()['items']
    else:
        return []

wine_query = "블로그에 나와있는 스테이크와 어울리는 와인과 기분 좋은 노래를 추천해주세요."
search_results = blog_search.invoke(wine_query)
search_results

[{'title': '이탈리아 피렌체 맛집 <b>추천</b> 티본<b>스테이크</b> 토스카나 와인 육회',
  'link': 'https://blog.naver.com/lsh5755/223847077153',
  'description': "피렌체 인근 토스카나 지방은 질 <b>좋은</b> <b>와인</b>을 생산하기로 유명하다. <b>와인</b> 애호가들의 성지기도 한다.... 그냥 'T본 <b>스테이크</b>'라고 부르고, 그들도 알아듣는다. T자 뼈가 포함되어 <b>있는</b> 안심과 등심 부위를... ",
  'bloggername': '천사는 여행중',
  'bloggerlink': 'blog.naver.com/lsh5755',
  'postdate': '20250426'},
 {'title': '<b>스테이크와</b> 잘 <b>어울리는</b> 3만원대 이마트 레드 <b>와인</b> <b>추천</b>',
  'link': 'https://blog.naver.com/dolmory9/223556006280',
  'description': '상태 <b>좋은</b> 코르크 : 산 페드로 시데랄 2021 ▲ <b>와인</b> 색상 : 산 페드로 시데랄 2021 (마트 토마스 글래스) &lt;음식 페어링&gt; 메인 육류 요리 또는 버섯과 잘 <b>어울리는</b> 레드 <b>와인</b>입니다. 등심 <b>스테이크와</b> 함께... ',
  'bloggername': '와인과 커피 엔지니어',
  'bloggerlink': 'blog.naver.com/dolmory9',
  'postdate': '20240821'},
 {'title': '여수 <b>와인</b>바 학동 맛집 :: <b>스테이크</b> 파스타 다이닝 바 와캡... ',
  'link': 'https://blog.naver.com/jhjoo20/223787083553',
  'description': '화이트 <b>와인</b>을 고를 때 직원분이 페어링하기 <b>좋은</b> 사이드로 <b>추천 

In [21]:
# LLM binding
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_blog = llm.bind_tools(tools=[tavily_search, blog_search])

In [22]:
# LLM 실행하여 툴 추천 받기
ai_msg = llm_with_blog.invoke(wine_query)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KlV2KNts3NZmhIfoW48Wch0o', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인"}', 'name': 'blog_search'}, 'type': 'function'}, {'id': 'call_SpT8c85QxCUK6l6AzCF0Qcwc', 'function': {'arguments': '{"query": "스테이크와 기분 좋은 노래"}', 'name': 'blog_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 131, 'total_tokens': 192, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BQYK1z6KCPqWoOjg2PkxLKCi9RsfP', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-539df725-ec42-4d5c-9fe7-2444f554932f-0', tool_calls=[{'name': 'blog_search', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_KlV2KNts3NZmhIfoW48Wch0

In [23]:
ai_msg.tool_calls

[{'name': 'blog_search',
  'args': {'query': '스테이크와 어울리는 와인'},
  'id': 'call_KlV2KNts3NZmhIfoW48Wch0o',
  'type': 'tool_call'},
 {'name': 'blog_search',
  'args': {'query': '스테이크와 기분 좋은 노래'},
  'id': 'call_SpT8c85QxCUK6l6AzCF0Qcwc',
  'type': 'tool_call'}]

In [24]:
# blog tool 실행
blog_search.batch(ai_msg.tool_calls)

[ToolMessage(content='[{"title": "스미스앤월렌스키 (<b>스테이크와 어울리는</b> 추천<b>와인</b>, 서울... ", "link": "https://blog.naver.com/neotaboo/223758018557", "description": "해주셔서 <b>와인</b>과 함께 <b>스테이크</b> 먹기 너무 좋은 매장이었다. 식전빵이 나왔다. 로즈메리를 가득 뿌려... 잘 <b>어울리는</b> 수프였다. 꾸덕한 식감도 좋고, 풍부한 크림맛도 좋고. 입 짧은 나도 처음으로 접시 전체를... ", "bloggername": "입 짧은 콩나물", "bloggerlink": "blog.naver.com/neotaboo", "postdate": "20250212"}, {"title": "<b>스테이크와 어울리는 와인</b> 추천과 종류", "link": "https://blog.naver.com/oipicle31/223708302326", "description": "Information <b>스테이크</b> <b>어울리는 와인</b> 추천 <b>스테이크와</b> <b>와인</b>, 이 둘의 조합은 마치 해변의 파도와 모래처럼 완벽하게 <b>어울리는</b> 궁합이라고 할 수 있죠. 오늘은 여러분의 저녁 식사를 더욱 특별하게 만들어줄... ", "bloggername": "별난하루", "bloggerlink": "blog.naver.com/oipicle31", "postdate": "20241229"}, {"title": "경주 황리단길 파스타 벚꽃과 정말 잘 <b>어울리는</b> <b>스테이크</b> <b>와인</b>... ", "link": "https://blog.naver.com/ljy213/223809691723", "description": "경주 황리단길 파스타 맛집 또봄은 이탈리아 음식점으로써 파스타와 <b>스테이크</b> 리조또 등의 메뉴가 주가 있었고, 다양한 <b>와인</b>이 있어서 그런지 메뉴판의 상당부분이 <b